In [5]:
import streamlit as st
import os
import openai
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, Tru

hugs = Huggingface()
openai = OpenAI()
tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


## Chain

In [2]:
template = """You are a chatbot having a conversation with a human.
        {chat_history}
        Human: {human_input}
        Chatbot:"""
    
prompt = PromptTemplate(input_variables=["chat_history", "human_input"], template=template)

memory = ConversationBufferMemory(memory_key="chat_history", max_memory_length=10)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

/home/biniyam/anaconda3/envs/fooocus-api/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### TruLenss Feedback Functions

In [9]:
f_relevance = Feedback(openai.relevance).on_input_output()


f_hate = Feedback(openai.moderation_hate).on_output()
f_violent = Feedback(openai.moderation_violence, higher_is_better=False).on_output()
f_selfharmed = Feedback(openai.moderation_selfharm, higher_is_better=False).on_output()
f_malice = Feedback(openai.maliciousness_with_cot_reasons, higher_is_better=False).on_output()

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In moderation_hate, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In moderation_violence, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In moderation_selfharm, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In maliciousness_with_cot_reasons, input text will be set to __record__.main_output or `Select.RecordOutput` .


In [10]:
chain_recorder = TruChain(
    chain, app_id='Simple RAG', feedbacks=[f_relevance, f_hate, f_violent, f_selfharmed, f_malice]
    
)

In [11]:
# Streamlit frontend
st.title("Contextual Chatbot")

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("What is up?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        # Record with TruLens
        with chain_recorder as recording:
            full_response = chain.run(prompt)
        message_placeholder = st.empty()
        message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append(
        {"role": "assistant", "content": full_response})

2024-02-20 11:42:03.107 
  command:

    streamlit run /home/biniyam/anaconda3/envs/fooocus-api/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-02-20 11:42:03.108 Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization